## Libraries Import and Install

In [24]:
!pip install ragas openai

In [25]:
import openai
import os
from google.colab import userdata
import pandas as pd
from tqdm import tqdm
import time


os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_KEY"
client = openai.OpenAI()

In [11]:
## Train Set
dataset_text = load_dataset('rag-datasets/rag-mini-wikipedia', 'text-corpus')

## Test Set
dataset_qa = load_dataset('rag-datasets/rag-mini-wikipedia', 'question-answer')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

data/passages.parquet/part.0.parquet:   0%|          | 0.00/797k [00:00<?, ?B/s]

Generating passages split:   0%|          | 0/3200 [00:00<?, ? examples/s]

data/test.parquet/part.0.parquet:   0%|          | 0.00/54.4k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/918 [00:00<?, ? examples/s]

In [13]:
## Converting to Dataframe for ease of use
df_text = pd.DataFrame.from_dict(dataset_text['passages'])

## Converting to Dataframe for ease of use
df_qa = pd.DataFrame.from_dict(dataset_qa['test'])

## RAGAS

#### Dataset

In [14]:
df_nrag = pd.read_csv('/content/drive/MyDrive/rag_data/predictions_prompts_181.csv')

In [15]:
df_erag = pd.read_csv('/content/drive/MyDrive/rag_data/persona_results_ARAG_181.csv')

In [16]:
df_nrag

,questions,true_answers,true_retrieved,persona
0,Was Abraham Lincoln the sixteenth President of...,yes,0,"Yes, Abraham Lincoln was the sixteenth Preside..."
1,Did Lincoln sign the National Banking Act of 1...,yes,2,"Yes, he did. The National Banking Act of 1863 ..."
2,Did his mother die of pneumonia?,no,4,No.\n\nExplanation: \n\nThis answer is based o...
3,How many long was Lincoln's formal education?,18 months,6,Lincoln had less than a year of formal educati...
4,When did Lincoln begin his political career?,1832,8,Lincoln began his political career in 1832. He...
...,...,...,...,...
175,Does Egypt have political influence in the Mid...,yes,397,"Yes, Egypt has extensive political influence i..."
176,When was the Six Day War?,1967,399,The Six Day War took place from June 5 to June...
177,What religions has Egypt outlawed?,"All but Christianity, Islam, and Judaism",401,"Egypt has outlawed Baha'i faith. In 1960, the ..."
178,What is the poulation of Egypt?,more than 78 million,403,Approximately 98 million people.


In [18]:
df_erag = df_erag[:180]

In [19]:
df_erag

,questions,true_answers,true_retrieved,persona
0,Was Abraham Lincoln the sixteenth President of...,yes,0,Yes\n\nExplanation:\n------------\n\nAbraham L...
1,Did Lincoln sign the National Banking Act of 1...,yes,2,"Yes, Lincoln signed the National Banking Act o..."
2,Did his mother die of pneumonia?,no,4,Yes\n```python\ndef answer_question(question):...
3,How many long was Lincoln's formal education?,18 months,6,"Lincoln was mostly self-educated, having very ..."
4,When did Lincoln begin his political career?,1832,8,"Lincoln's political career began in 1832, when..."
...,...,...,...,...
175,Does Egypt have political influence in the Mid...,yes,397,"Yes, Egypt has political influence in the Midd..."
176,When was the Six Day War?,1967,399,The Six Day War took place in June 1967. The w...
177,What religions has Egypt outlawed?,"All but Christianity, Islam, and Judaism",401,Egypt has outlawed the practice of Baha'i fait...
178,What is the poulation of Egypt?,more than 78 million,403,The population of Egypt is approximately 101 m...


#### Evaluation

OpenAI

In [47]:
def evaluate_answer_comprehensive(question, llm_answer, ground_truth, model="gpt-4o-mini"):
    """
    Evaluate LLM answer comprehensively using GPT-4o-mini:
    1. Correctness - Does the LLM answer contain the correct information from ground truth?
    2. Faithfulness - Is the answer factually accurate (no hallucinations)?
    3. Helpfulness - Is the answer relevant and helpful?
    """

    prompt = f"""Evaluate the LLM's answer against the ground truth. The ground truth is brief, but the LLM answer is detailed. Judge if the LLM answer is correct even if more verbose.

Question: {question}
Ground Truth: {ground_truth}
LLM Answer: {llm_answer}

Evaluate on three criteria:
1. CORRECTNESS: Does the LLM answer contain the same factual information as the ground truth? Even if more detailed, is the core answer correct? (Correct/Incorrect)
2. FAITHFULNESS: Is the LLM answer factually accurate with no made-up information or hallucinations? (Faithful/Unfaithful)
3. HELPFULNESS: Is the answer helpful and relevant to the question? (Helpful/Unhelpful)

Respond EXACTLY as: [Correct/Incorrect], [Faithful/Unfaithful], [Helpful/Unhelpful]

Your response:"""

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_completion_tokens=20  # Changed from max_tokens
    )

    return response.choices[0].message.content.strip()

In [48]:
def parse_evaluation(eval_string):
    """Parse the comma-separated response"""
    try:
        parts = [x.strip() for x in eval_string.split(',')]
        if len(parts) == 3:
            return {
                'correctness': parts[0],
                'faithfulness': parts[1],
                'helpfulness': parts[2]
            }
        else:
            return {
                'correctness': 'Error',
                'faithfulness': 'Error',
                'helpfulness': 'Error'
            }
    except:
        return {
            'correctness': 'Error',
            'faithfulness': 'Error',
            'helpfulness': 'Error'
        }

In [49]:
def evaluate_with_retry(question, llm_answer, ground_truth, max_retries=3):
    """Evaluate with retry for API errors"""
    for attempt in range(max_retries):
        try:
            eval_result = evaluate_answer_comprehensive(question, llm_answer, ground_truth)
            return parse_evaluation(eval_result)
        except Exception as e:
            print(f"Error on attempt {attempt + 1}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return {
                    'correctness': 'Error',
                    'faithfulness': 'Error',
                    'helpfulness': 'Error'
                }

True Evaluation

Enhanced RAG

In [51]:
## CSV Copy
df = df_erag.copy()

# Evaluate all rows
print("\nStarting evaluation...")
results = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    question = row['questions']
    ground_truth = str(row['true_answers'])  # Convert to string in case of numbers
    llm_answer = str(row['persona'])

    scores = evaluate_with_retry(question, llm_answer, ground_truth)

    results.append({
        'correctness': scores['correctness'],
        'faithfulness': scores['faithfulness'],
        'helpfulness': scores['helpfulness']
    })
    time.sleep(0.3)

# Add evaluation columns to dataframe
df['correctness'] = [r['correctness'] for r in results]
df['faithfulness'] = [r['faithfulness'] for r in results]
df['helpfulness'] = [r['helpfulness'] for r in results]

# Calculate metrics
total = len(df)
correct_count = (df['correctness'] == 'Correct').sum()
faithful_count = (df['faithfulness'] == 'Faithful').sum()
helpful_count = (df['helpfulness'] == 'Helpful').sum()
error_count = (df['correctness'] == 'Error').sum()

# Perfect answers (all three criteria met)
perfect_count = ((df['correctness'] == 'Correct') &
                 (df['faithfulness'] == 'Faithful') &
                 (df['helpfulness'] == 'Helpful')).sum()

print("\n" + "="*70)
print("COMPREHENSIVE EVALUATION RESULTS")
print("="*70)
print(f"Total Questions Evaluated: {total}")
print(f"Errors: {error_count}")

print(f"\n{'CORRECTNESS (vs Ground Truth):':<40}")
print(f"  Correct:   {correct_count:>4} ({correct_count/total*100:>5.1f}%)")
print(f"  Incorrect: {total - correct_count - error_count:>4} ({(total-correct_count-error_count)/total*100:>5.1f}%)")

print(f"\n{'FAITHFULNESS (No Hallucinations):':<40}")
print(f"  Faithful:   {faithful_count:>4} ({faithful_count/total*100:>5.1f}%)")
print(f"  Unfaithful: {total - faithful_count - error_count:>4} ({(total-faithful_count-error_count)/total*100:>5.1f}%)")

print(f"\n{'HELPFULNESS:':<40}")
print(f"  Helpful:   {helpful_count:>4} ({helpful_count/total*100:>5.1f}%)")
print(f"  Unhelpful: {total - helpful_count - error_count:>4} ({(total-helpful_count-error_count)/total*100:>5.1f}%)")

print(f"\n{'PERFECT ANSWERS (All 3 Criteria):':<40}")
print(f"  {perfect_count:>4} ({perfect_count/total*100:>5.1f}%)")

# Save results
output_filename = '/content/drive/MyDrive/rag_data/ERAG_evaluation_results_with_scores.csv'
df.to_csv(output_filename, index=False)
print(f"\n✓ Results saved to '{output_filename}'")


Starting evaluation...


100%|██████████| 180/180 [05:38<00:00,  1.88s/it]


COMPREHENSIVE EVALUATION RESULTS
Total Questions Evaluated: 180
Errors: 0

CORRECTNESS (vs Ground Truth):          
  Correct:    113 ( 62.8%)
  Incorrect:   67 ( 37.2%)

FAITHFULNESS (No Hallucinations):       
  Faithful:    120 ( 66.7%)
  Unfaithful:   60 ( 33.3%)

HELPFULNESS:                            
  Helpful:    118 ( 65.6%)
  Unhelpful:   62 ( 34.4%)

PERFECT ANSWERS (All 3 Criteria):       
   110 ( 61.1%)

✓ Results saved to '/content/drive/MyDrive/rag_data/ERAG_evaluation_results_with_scores.csv'


Naive RAG

In [52]:
## CSV Copy
df = df_nrag.copy()

# Evaluate all rows
print("\nStarting evaluation...")
results = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    question = row['questions']
    ground_truth = str(row['true_answers'])  # Convert to string in case of numbers
    llm_answer = str(row['persona'])

    scores = evaluate_with_retry(question, llm_answer, ground_truth)

    results.append({
        'correctness': scores['correctness'],
        'faithfulness': scores['faithfulness'],
        'helpfulness': scores['helpfulness']
    })
    time.sleep(0.3)

# Add evaluation columns to dataframe
df['correctness'] = [r['correctness'] for r in results]
df['faithfulness'] = [r['faithfulness'] for r in results]
df['helpfulness'] = [r['helpfulness'] for r in results]

# Calculate metrics
total = len(df)
correct_count = (df['correctness'] == 'Correct').sum()
faithful_count = (df['faithfulness'] == 'Faithful').sum()
helpful_count = (df['helpfulness'] == 'Helpful').sum()
error_count = (df['correctness'] == 'Error').sum()

# Perfect answers (all three criteria met)
perfect_count = ((df['correctness'] == 'Correct') &
                 (df['faithfulness'] == 'Faithful') &
                 (df['helpfulness'] == 'Helpful')).sum()

print("\n" + "="*70)
print("COMPREHENSIVE EVALUATION RESULTS")
print("="*70)
print(f"Total Questions Evaluated: {total}")
print(f"Errors: {error_count}")

print(f"\n{'CORRECTNESS (vs Ground Truth):':<40}")
print(f"  Correct:   {correct_count:>4} ({correct_count/total*100:>5.1f}%)")
print(f"  Incorrect: {total - correct_count - error_count:>4} ({(total-correct_count-error_count)/total*100:>5.1f}%)")

print(f"\n{'FAITHFULNESS (No Hallucinations):':<40}")
print(f"  Faithful:   {faithful_count:>4} ({faithful_count/total*100:>5.1f}%)")
print(f"  Unfaithful: {total - faithful_count - error_count:>4} ({(total-faithful_count-error_count)/total*100:>5.1f}%)")

print(f"\n{'HELPFULNESS:':<40}")
print(f"  Helpful:   {helpful_count:>4} ({helpful_count/total*100:>5.1f}%)")
print(f"  Unhelpful: {total - helpful_count - error_count:>4} ({(total-helpful_count-error_count)/total*100:>5.1f}%)")

print(f"\n{'PERFECT ANSWERS (All 3 Criteria):':<40}")
print(f"  {perfect_count:>4} ({perfect_count/total*100:>5.1f}%)")

# Save results
output_filename = '/content/drive/MyDrive/rag_data/NRAG_evaluation_results_with_scores.csv'
df.to_csv(output_filename, index=False)
print(f"\n✓ Results saved to '{output_filename}'")


Starting evaluation...


100%|██████████| 180/180 [02:51<00:00,  1.05it/s]


COMPREHENSIVE EVALUATION RESULTS
Total Questions Evaluated: 180
Errors: 0

CORRECTNESS (vs Ground Truth):          
  Correct:     74 ( 41.1%)
  Incorrect:  106 ( 58.9%)

FAITHFULNESS (No Hallucinations):       
  Faithful:     80 ( 44.4%)
  Unfaithful:  100 ( 55.6%)

HELPFULNESS:                            
  Helpful:     78 ( 43.3%)
  Unhelpful:  102 ( 56.7%)

PERFECT ANSWERS (All 3 Criteria):       
    72 ( 40.0%)

✓ Results saved to '/content/drive/MyDrive/rag_data/NRAG_evaluation_results_with_scores.csv'
